In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
def generate_boxplot_from_list(data: list, title: str) -> None:
    """
    Muestra un boxplot horizontal para visualizar anomalías en precios.

    Parámetros:
    data : list
        Lista de valores numéricos a graficar.
    """
    plt.figure(figsize=(8, 2))
    plt.boxplot(data, vert=False)
    plt.grid(True)
    plt.xlabel("value")
    plt.title(title)
    plt.show()



In [ ]:
def plot_corr_with_target(df: pd.DataFrame, target: str = 'quality') -> None:
    """
    Plots a horizontal barplot showing the absolute correlation of all numeric features with the target variable.

    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing features and the target.
    target : str
        The name of the target column to correlate against. Defaults to 'quality'.
    """
    corrs = df.corr(numeric_only=True)[target].drop(target).sort_values(key=abs, ascending=False)
    plt.figure(figsize=(10, 4))
    sns.barplot(x=corrs.values, y=corrs.index, palette='coolwarm')
    plt.title(f'Correlation with {target}')
    plt.xlabel('Correlation')
    plt.grid(True, axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


In [ ]:
def evaluar_modelos_regresion(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    usar_pesos: bool = False
) -> tuple[np.ndarray, np.ndarray, float, float]:
    """
    Entrena y evalúa dos modelos de regresión (LinearRegression y GradientBoostingRegressor)
    usando RMSE como métrica. Opcionalmente aplica pesos balanceados por clase.

    Parámetros:
    ----------
    X_train : pd.DataFrame. Conjunto de entrenamiento (features).
    X_test : pd.DataFrame. Conjunto de prueba (features).
    y_train : pd.Series. Variable objetivo de entrenamiento.
    y_test : pd.Series. Variable objetivo de prueba.
    usar_pesos : bool, opcional. Si es True, utiliza pesos balanceados por clase (por defecto: False).

    Retorna:
    -------
    y_pred_lr : np.ndarray. Predicciones del modelo LinearRegression.
    y_pred_gbr : np.ndarray. Predicciones del modelo GradientBoostingRegressor.
    rmse_lr : float. RMSE del modelo LinearRegression.
    rmse_gbr : float. RMSE del modelo GradientBoostingRegressor.
    """
    sample_weight = None
    if usar_pesos:
        sample_weight = compute_sample_weight(class_weight='balanced', y=y_train.astype(int))

    lr = LinearRegression()
    gbr = GradientBoostingRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

    lr.fit(X_train, y_train, sample_weight=sample_weight)
    gbr.fit(X_train, y_train, sample_weight=sample_weight)

    y_pred_lr = lr.predict(X_test)
    y_pred_gbr = gbr.predict(X_test)

    rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
    rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))

    print(f"RMSE logistic regression: {rmse_lr:.3f}")
    print(f"RMSE gradient boosting:   {rmse_gbr:.3f}")

    return y_pred_lr, y_pred_gbr, rmse_lr, rmse_gbr


In [ ]:
def extraer_metricas(y_pred: np.ndarray, y_true: pd.Series, modelo: str) -> None:
    """
    Calcula e imprime métricas de clasificación (accuracy, precision, recall y F1-score) 
    tanto globales como por clase. Asume que y_pred proviene de un modelo regresor 
    y por lo tanto debe ser redondeado.

    Parámetros:
    ----------
    y_pred : np.ndarray. Predicciones del modelo (valores continuos a redondear).
    y_true : pd.Series. Valores reales de la variable objetivo.
    modelo : str. Nombre del modelo a mostrar en los encabezados.
    """
    y_pred_int = np.round(y_pred).astype(int)
    y_true_int = y_true.astype(int)

    acc = accuracy_score(y_true_int, y_pred_int)

    print(f"\n=== Métricas globales para {modelo} ===")
    print(f"Accuracy: {100 * acc:.2f} %")    
    print(f"Precision macro:    {precision_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print(f"Recall macro:       {recall_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print(f"F1 macro:           {f1_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print("\n--- Métricas por clase ---")
    clases = np.unique(y_true_int)
    prec_none = precision_score(y_true_int, y_pred_int, average=None)
    rec_none = recall_score(y_true_int, y_pred_int, average=None)
    f1_none = f1_score(y_true_int, y_pred_int, average=None)

    for i, c in enumerate(clases):
        print(f"Clase {c}: Precision = {prec_none[i]:.3f}, Recall = {rec_none[i]:.3f}, F1 = {f1_none[i]:.3f}")


In [ ]:
def entrenar_gradient_boosting_CV(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    grid: dict
) -> tuple[np.ndarray, float]:
    """
    Entrena un modelo Gradient Boosting con búsqueda de hiperparámetros mediante GridSearchCV
    y evaluación usando RMSE. Utiliza sample weights balanceados por clase.

    Parámetros:
    ----------
    X_train : pd.DataFrame. Conjunto de entrenamiento (features).
    X_test : pd.DataFrame. Conjunto de prueba (features).
    y_train : pd.Series. Variable objetivo de entrenamiento.
    y_test : pd.Series. Variable objetivo de prueba.
    grid : dict. Diccionario con los hiperparámetros a explorar en el GridSearch.

    Retorna:
    -------
    y_pred : np.ndarray. Predicciones del modelo optimizado.
    rmse : float. Raíz del error cuadrático medio sobre el conjunto de test.
    """
    sample_weight = compute_sample_weight(class_weight='balanced', y=y_train.astype(int))

    gbr_base = GradientBoostingRegressor(random_state=42)
    gbr = GridSearchCV(
        estimator=gbr_base,
        param_grid=grid,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=-1
    )

    gbr.fit(X_train, y_train, sample_weight=sample_weight)
    y_pred = gbr.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"RMSE Gradient Boosting: {rmse:.3f}")
    print(f"Mejores parámetros: {gbr.best_params_}")

    return y_pred, rmse


In [ ]:
def plot_confusion_matrix(
    y_pred: np.ndarray,
    y_true: pd.Series,
    titulo: str = "Matriz de confusión"
) -> None:
    """
    Genera y muestra una matriz de confusión a partir de predicciones continuas redondeadas.

    Parámetros:
    ----------
    y_pred : np.ndarray. Predicciones del modelo (valores continuos que serán redondeados a enteros).
    y_true : pd.Series. Valores reales de la variable objetivo.
    titulo : str, opcional. Título del gráfico de la matriz de confusión. Por defecto: "Matriz de confusión".
    """
    y_pred_clasificado = np.round(y_pred).astype(int)
    y_true_clasificado = y_true.astype(int)

    cm = confusion_matrix(y_true_clasificado, y_pred_clasificado)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Oranges', xticks_rotation=45)
    plt.title(titulo)
    plt.show()


In [ ]:
def evaluar_modelos_clasificacion(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    usar_pesos: bool = False
) -> tuple[np.ndarray, np.ndarray]:
    """
    Entrena modelos de clasificación (Logistic Regression y Gradient Boosting Classifier)
    y devuelve sus predicciones sobre el conjunto de test.

    Parámetros:
    ----------
    X_train : pd.DataFrame. Conjunto de entrenamiento (features).
    X_test : pd.DataFrame. Conjunto de prueba (features).
    y_train : pd.Series. Variable objetivo de entrenamiento.
    usar_pesos : bool, opcional. Si es True, utiliza sample weights balanceados. Por defecto es False.

    Retorna:
    -------
    lr_pred : np.ndarray. Predicciones del modelo Logistic Regression.
    gb_pred : np.ndarray. Predicciones del modelo Gradient Boosting Classifier.
    """
    sample_weight = None
    if usar_pesos:
        sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)

    lr = LogisticRegression(max_iter=1000, random_state=42)
    gbc = GradientBoostingClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

    lr.fit(X_train, y_train, sample_weight=sample_weight)
    gbc.fit(X_train, y_train, sample_weight=sample_weight)

    lr_pred = lr.predict(X_test)
    gb_pred = gbc.predict(X_test)
    return lr_pred, gb_pred


def extraer_metricas_clasificacion(
    y_pred: np.ndarray,
    y_true: pd.Series,
    modelo: str
) -> None:
    """
    Imprime métricas de evaluación de clasificación: accuracy, precision, recall y F1,
    tanto globales (macro) como por clase (None).

    Parámetros:
    ----------
    y_pred : np.ndarray. Predicciones del modelo.
    y_true : pd.Series. Valores reales de la variable objetivo.
    modelo : str. Nombre del modelo (para mostrar en el encabezado).
    """
    y_pred_int = y_pred.astype(int)
    y_true_int = y_true.astype(int)

    acc = accuracy_score(y_true_int, y_pred_int)

    print(f"\n=== Métricas globales para {modelo} ===")
    print(f"Accuracy: {100 * acc:.2f} %")    
    print(f"Precision macro:    {precision_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print(f"Recall macro:       {recall_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print(f"F1 macro:           {f1_score(y_true_int, y_pred_int, average='macro'):.3f}")
    print("\n--- Métricas por clase ---")
    clases = np.unique(y_true_int)
    prec_none = precision_score(y_true_int, y_pred_int, average=None)
    rec_none = recall_score(y_true_int, y_pred_int, average=None)
    f1_none = f1_score(y_true_int, y_pred_int, average=None)

    for i, c in enumerate(clases):
        print(f"Clase {c}: Precision = {prec_none[i]:.3f}, Recall = {rec_none[i]:.3f}, F1 = {f1_none[i]:.3f}")


In [ ]:
def entrenar_gradient_boosting_CV_clasificacion(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    grid: dict
) -> tuple[np.ndarray, float]:
    """
    Entrena un modelo Gradient Boosting Classifier usando GridSearchCV para encontrar los mejores
    hiperparámetros. Evalúa el modelo en el conjunto de test utilizando accuracy y muestra los
    mejores parámetros encontrados. Usa sample weights balanceados.

    Parámetros:
    ----------
    X_train : pd.DataFrame. Conjunto de entrenamiento (features).
    X_test : pd.DataFrame. Conjunto de prueba (features).
    y_train : pd.Series. Variable objetivo de entrenamiento.
    y_test : pd.Series. Variable objetivo de prueba.
    grid : dict. Diccionario con los hiperparámetros a explorar en el GridSearch.

    Retorna:
    -------
    y_pred : np.ndarray. Predicciones del mejor modelo sobre el conjunto de test.
    acc : float. Accuracy obtenido en el conjunto de test.
    """
    sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)

    gbc_base = GradientBoostingClassifier(random_state=42)
    gbc = GridSearchCV(
        estimator=gbc_base,
        param_grid=grid,
        scoring='accuracy',
        cv=5,
        n_jobs=-1
    )

    gbc.fit(X_train, y_train, sample_weight=sample_weight)
    y_pred = gbc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"Accuracy Gradient Boosting: {100 * acc:.2f} %")
    print(f"Mejores parámetros: {gbc.best_params_}")

    return y_pred, acc
